In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Sickle_Cell_Anemia"
cohort = "GSE53441"

# Input paths
in_trait_dir = "../../input/GEO/Sickle_Cell_Anemia"
in_cohort_dir = "../../input/GEO/Sickle_Cell_Anemia/GSE53441"

# Output paths
out_data_file = "../../output/preprocess/Sickle_Cell_Anemia/GSE53441.csv"
out_gene_data_file = "../../output/preprocess/Sickle_Cell_Anemia/gene_data/GSE53441.csv"
out_clinical_data_file = "../../output/preprocess/Sickle_Cell_Anemia/clinical_data/GSE53441.csv"
json_path = "../../output/preprocess/Sickle_Cell_Anemia/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Associations of inflammation, iron and early death in sickle cell disease"
!Series_summary	"Expression profiling using a defined set of iron regulated genes identifies co-regulation of genes and pathways related to inflammatory cytokines"
!Series_summary	"In this dataset, we include the whole genome expression data obtained from PBMCs from steady state HbSSs with Sickle Cell Disease and healthy  HbAAs . Unsupervised two-way hierarchical clustering using a predefined iron regulated gene list identified 3 groups of subjects with high, low and intermediate expression of iron activated genes.In analysis of these three iron regulated gene clusters, we found 98 genes which were consistently and significantly differentially expressed."
!Series_overall_design	"Samples from 24 subjects with SCA and 9 healthy subjects were analysed. With approval from the institutional review board, RNA was isolated from PBMCs and plasma was stored for analysis of baseline 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Callable, Dict, Any, Optional

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data from Affymetrix Human Genome microarray
is_gene_available = True

# 2.1 Data Availability
# From the sample characteristics dictionary, we can identify relevant rows:
# trait_row = 0 (diagnosis: sickle cell anemia (SCA) or diagnosis: normal)
# There's no age data available
# There's no gender data available
trait_row = 0
age_row = None
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert trait value to binary (0 for normal, 1 for SCA)"""
    if pd.isna(value) or value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary
    if 'sickle cell anemia' in value.lower() or 'sca' in value.lower():
        return 1
    elif 'normal' in value.lower():
        return 0
    else:
        return None

def convert_age(value):
    """Convert age value to continuous"""
    # Not used since age data is not available
    return None

def convert_gender(value):
    """Convert gender value to binary (0 for female, 1 for male)"""
    # Not used since gender data is not available
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Conduct initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Assuming clinical_data is already available from previous step
    try:
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,  # Use existing variable
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the dataframe
        preview = preview_df(selected_clinical_df)
        print("Preview of selected clinical features:")
        for key, values in preview.items():
            print(f"{key}: {values}")
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save the clinical data
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except NameError:
        print("Warning: clinical_data variable is not available. Cannot extract clinical features.")
    except Exception as e:
        print(f"Error during clinical feature extraction: {str(e)}")


Preview of selected clinical features:
GSM1293592: [1.0]
GSM1293593: [1.0]
GSM1293594: [1.0]
GSM1293595: [1.0]
GSM1293596: [1.0]
GSM1293597: [1.0]
GSM1293598: [1.0]
GSM1293599: [1.0]
GSM1293600: [1.0]
GSM1293601: [1.0]
GSM1293602: [1.0]
GSM1293603: [1.0]
GSM1293604: [1.0]
GSM1293605: [1.0]
GSM1293606: [1.0]
GSM1293607: [1.0]
GSM1293608: [1.0]
GSM1293609: [1.0]
GSM1293610: [1.0]
GSM1293611: [1.0]
GSM1293612: [1.0]
GSM1293613: [1.0]
GSM1293614: [1.0]
GSM1293615: [1.0]
GSM1293616: [0.0]
GSM1293617: [0.0]
GSM1293618: [0.0]
GSM1293619: [0.0]
GSM1293620: [0.0]
GSM1293621: [0.0]
GSM1293622: [0.0]
GSM1293623: [0.0]
GSM1293624: [0.0]
GSM1293625: [0.0]
Clinical data saved to ../../output/preprocess/Sickle_Cell_Anemia/clinical_data/GSE53441.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 61
Header line: "ID_REF"	"GSM1293592"	"GSM1293593"	"GSM1293594"	"GSM1293595"	"GSM1293596"	"GSM1293597"	"GSM1293598"	"GSM1293599"	"GSM1293600"	"GSM1293601"	"GSM1293602"	"GSM1293603"	"GSM1293604"	"GSM1293605"	"GSM1293606"	"GSM1293607"	"GSM1293608"	"GSM1293609"	"GSM1293610"	"GSM1293611"	"GSM1293612"	"GSM1293613"	"GSM1293614"	"GSM1293615"	"GSM1293616"	"GSM1293617"	"GSM1293618"	"GSM1293619"	"GSM1293620"	"GSM1293621"	"GSM1293622"	"GSM1293623"	"GSM1293624"	"GSM1293625"
First data line: "1007_s_at"	6.26	6.62	6.36	6.44	6.67	6.48	6.36	6.82	6.35	6.44	6.4	6.27	6.02	6.55	6.71	6.81	6.95	7.21	6.33	6.62	6.6	6.28	6.51	6.91	6.27	6.11	6.47	6.28	6.47	6.35	6.59	6.35	6.5	5.94
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='I

### Step 4: Gene Identifier Review

In [5]:
# Examining the identifiers in the gene expression data
# The identifiers appear to be Affymetrix probe IDs (e.g., '1007_s_at', '1053_at', etc.)
# These are not standard human gene symbols but rather microarray probe identifiers
# They need to be mapped to standard gene symbols for biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE53441
Line 6: !Series_title = Associations of inflammation, iron and early death in sickle cell disease
Line 7: !Series_geo_accession = GSE53441
Line 8: !Series_status = Public on Dec 31 2014
Line 9: !Series_submission_date = Dec 18 2013
Line 10: !Series_last_update_date = Mar 25 2019
Line 11: !Series_pubmed_id = 25378535
Line 12: !Series_summary = Expression profiling using a defined set of iron regulated genes identifies co-regulation of genes and pathways related to inflammatory cytokines
Line 13: !Series_summary = In this dataset, we include the whole genome expression data obtained from PBMCs from steady state HbSSs with Sickle Cell Disease and healthy  HbAAs . Unsupervised two-way hierarchical clus


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DE

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the relevant columns from the gene annotation data
# From previous output, we can see that:
# - 'ID' column contains the probe identifiers (e.g., '1007_s_at')
# - 'Gene Symbol' column contains the gene symbols (e.g., 'DDR1 /// MIR4640')

# Get the mapping dataframe from the annotation data
prob_col = 'ID'  # Column with probe identifiers
gene_col = 'Gene Symbol'  # Column with gene symbols

# 2. Create the mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"Mapping dataframe shape: {mapping_df.shape}")
print("Sample of mapping data:")
print(mapping_df.head())

# 3. Apply gene mapping to convert probe-level measurements to gene-level expression
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First few gene symbols after mapping:")
print(gene_data.index[:10])

# 4. Save the gene expression data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Mapping dataframe shape: (45782, 2)
Sample of mapping data:
          ID              Gene
0  1007_s_at  DDR1 /// MIR4640
1    1053_at              RFC2
2     117_at             HSPA6
3     121_at              PAX8
4  1255_g_at            GUCA1A
Gene expression data shape after mapping: (21278, 34)
First few gene symbols after mapping:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A4GALT', 'A4GNT', 'AA06'],
      dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Sickle_Cell_Anemia/gene_data/GSE53441.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
print(f"Gene data shape before normalization: {gene_data.shape}")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load and prepare clinical features
try:
    # Load the clinical features created in a previous step
    clinical_features = pd.read_csv(out_clinical_data_file)
    print(f"Loaded clinical features with shape: {clinical_features.shape}")
    
    # Rename the column to the trait name if it's a numeric column
    if '0' in clinical_features.columns:
        clinical_features = clinical_features.rename(columns={'0': trait})
    
    # Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
    print(f"Linked data shape: {linked_data.shape}")
    print("Linked data preview (first 5 rows, first 5 columns):")
    print(linked_data.iloc[:5, :5])
    
    # 3. Handle missing values
    print("Missing values before handling:")
    print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
    if 'Age' in linked_data.columns:
        print(f"  Age missing: {linked_data['Age'].isna().sum()} out of {len(linked_data)}")
    if 'Gender' in linked_data.columns:
        print(f"  Gender missing: {linked_data['Gender'].isna().sum()} out of {len(linked_data)}")
    
    gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
    print(f"  Genes with >20% missing: {sum(linked_data[gene_cols].isna().mean() > 0.2)}")
    print(f"  Samples with >5% missing genes: {sum(linked_data[gene_cols].isna().mean(axis=1) > 0.05)}")
    
    cleaned_data = handle_missing_values(linked_data, trait)
    print(f"Data shape after handling missing values: {cleaned_data.shape}")
    
    # 4. Evaluate bias in trait and demographic features
    is_trait_biased = False
    if len(cleaned_data) > 0:
        trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
        is_trait_biased = trait_biased
    else:
        print("No data remains after handling missing values.")
        is_trait_biased = True
    
    # 5. Final validation and save
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=True, 
        is_biased=is_trait_biased, 
        df=cleaned_data,
        note="Dataset includes sickle cell anemia patients and healthy controls, processed from Affymetrix microarray data."
    )
    
    # 6. Save if usable
    if is_usable and len(cleaned_data) > 0:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        cleaned_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Data was determined to be unusable or empty and was not saved")
        
except Exception as e:
    print(f"Error processing data: {e}")
    # Handle the error case by still recording cohort info
    validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=False,  # Mark as not available due to processing issues
        is_biased=True, 
        df=pd.DataFrame(),  # Empty dataframe
        note=f"Error processing data for {trait}: {str(e)}"
    )
    print("Data was determined to be unusable and was not saved")

Gene data shape before normalization: (21278, 34)
Gene data shape after normalization: (19845, 34)


Normalized gene data saved to ../../output/preprocess/Sickle_Cell_Anemia/gene_data/GSE53441.csv
Loaded clinical features with shape: (1, 34)
Linked data shape: (34, 19846)
Linked data preview (first 5 rows, first 5 columns):
              0  A1BG  A1BG-AS1  A1CF   A2M
GSM1293592  1.0  6.79      4.73  8.22  8.01
GSM1293593  1.0  6.20      4.52  8.20  7.70
GSM1293594  1.0  6.58      4.65  8.43  7.94
GSM1293595  1.0  6.35      4.62  7.87  7.27
GSM1293596  1.0  6.24      4.21  8.07  8.18
Missing values before handling:
Error processing data: 'Sickle_Cell_Anemia'
Abnormality detected in the cohort: GSE53441. Preprocessing failed.
Data was determined to be unusable and was not saved
